### 5장. 웹 크롤링을 이용한 데이터 수집

5.3 [프로젝트] 증권사 리포트 수집

<u>파이썬을 이용한 리포트 수집</u>

In [1]:
import requests
from bs4 import BeautifulSoup as BS

In [2]:
url = 'http://www.paxnet.co.kr/stock/report/report?menuCode=2222'
response = requests.get(url)
html = response.content
html = BS(html)

In [3]:
div = html.find('div', attrs={"class": "board-type"})
li_list = div.find_all('li')

<u>수집 대상 행 결정</u>

In [4]:
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [5]:
print(li_list[1])

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=115180">큐리언트</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('142510');">정상화 진행 중</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
										
											-원
										
									
								</div>
<div class="line3 color-black">
									-
								</div>
<div class="line3">한국IR협의회</div>
<div class="line3">2023.08.18</div>
</li>


In [6]:
print(li_list[-1])

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>


In [10]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [11]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

True
False
False


In [12]:
stock = li_list[1].find_all('a')[0].text
print(stock)

큐리언트


In [13]:
title = li_list[1].find_all('a')[1].text
print(title)

정상화 진행 중


In [17]:
price = li_list[4].find('div', attrs={'class': 'line3'}).text
display(price)

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t102,000원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

In [18]:
price = price.replace('\r', '').replace('\t', '').replace('\n', '')
price = price.split(' ')[1].replace(',', '')[:-1]
display(price)

'102000'

In [21]:
opinion = li_list[5].find_all('div', attrs={'class': 'line3'})[1].text
display(opinion)

'\r\n\t\t\t\t\t\t\t\t\t매수(유지)\r\n\t\t\t\t\t\t\t\t'

In [22]:
opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
display(opinion)

'매수(유지)'

In [23]:
trading_firm = li_list[1].find_all('div', attrs={'class': 'line3'})[2].text
display(trading_firm)

'한국IR협의회'

In [25]:
date = li_list[1].find_all('div', attrs={'class': 'line3'})[3].text
display(date)

'2023.08.18'

In [26]:
import pandas as pd

date = pd.to_datetime(date)
display(date)

Timestamp('2023-08-18 00:00:00')

<u>단일 리포트 파싱</u>

In [27]:
def parsing_li(li):
    stock = li.find_all('a')[0].text # 종목
    title = li.find_all('a')[1].text # 리포트 종목
    price = li.find('div', attrs={'class': 'line3'}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all('div', attrs={'class': 'line3'})[1].text # 투자의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all('div', attrs={'class': 'line3'})[2].text # 증권사
    date = li.find_all('div', attrs={'class': 'line3'})[3].text # 작성일
    date = pd.to_datetime(date)

    return [stock, title, price, opinion, trading_firm, date]

In [28]:
print(parsing_li(li_list[1]))

['큐리언트', '정상화 진행 중', '-', '-', '한국IR협의회', Timestamp('2023-08-18 00:00:00')]


<u>한 페이지 내 모든 리포트 수집 및 정제</u>

In [29]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)

data = pd.DataFrame(data, columns=['종목', '리포트제목', '적정가격', '투자의견','증권사', '작성일'])

In [30]:
display(data.head(10))

,종목,리포트제목,적정가격,투자의견,증권사,작성일
0,큐리언트,정상화 진행 중,-,-,한국IR협의회,2023-08-18
1,LX하우시스,"2023년 영업이익 6년만에 1,000억원 상회 전망",-,-,한국IR협의회,2023-08-18
2,NHN KCP,가맹점 확대에 따른 지속적인 매출 성장 기대,-,-,한국IR협의회,2023-08-18
3,대한유화,"2분기 영업손실 예상 상회, 당분간 적자 지속 전망",102000,시장수익률,키움증권,2023-08-17
4,SK오션플랜트,2분기 사상 최대 실적,30000,매수(유지),키움증권,2023-08-17
5,삼성전기,전장 부품 업체로 거듭나기,200000,매수(유지),키움증권,2023-08-17
6,영원무역,2Q23R: 헤비급 실적 vs. 경량급 주가,,매수,메리츠증권,2023-08-17
7,SK오션플랜트,2Q23 Review: 탑라인 성장 한계를 벗겨내다,,매수,메리츠증권,2023-08-17
8,JYP Ent.,기대에 못 미쳐도 여전히 좋은 실적,155000,매수,메리츠증권,2023-08-17
9,노바텍,3분기 성수기 진입 + 신사업 기대감 유효,-,-,메리츠증권,2023-08-16


<u>전체 페이지의 데이터 수집 및 파싱</u>

In [33]:
import time

base_url = 'http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue='
data = []

for page_no in range(1, 100): # 99페이지 까지 가져오기
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)

    html = response.content
    html = BS(html)

    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find('div', attrs={'class': 'board-type'})
    li_list = div.find_all('li')
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns=['종목', '리포트제목', '적정가격', '투자의견','증권사', '작성일'])

<u>데이터 저장</u>

In [38]:
data.to_csv('./data/stock_report.txt', sep='\t', encoding='utf-8', index=False)